In [ ]:
# 1 «Звёздные войны». Сохраните файл в формате YAML.


In [ ]:
# 2 server.json Прочитайте файл и внесите изменения.


In [ ]:
# 3 файл с логами sshd. Постройте гистограмму неудачных входов в зависимости от времени, с размером корзины 5–10 минут.


In [ ]:
# 4 файл Is.txt c помощью регулярных выражений посчитайте суммарный размер файлов


In [ ]:
# 5 база данных в архиве. Выведите список всех клиентов, которые проживают в городе, начинающемся на букву А


In [ ]:
# 6 Воспользуйтесь кодом из раздела «Рекомендация» для создания базы данных taxis.sqlite3.
